###  Retrieval-Augmented Generation (RAG) for Inventory Scoring

#### Import Packages

In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join('..')))

In [2]:
from rag.rag_pipeline import Rag
import pandas as pd

/home/babi/miniconda3/envs/week10/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
2024-03-21 15:51:56,535 - DEBUG - load_ssl_context verify=True cert=None trust_env=True http2=False
2024-03-21 15:51:56,537 - DEBUG - load_verify_locations cafile='/home/babi/miniconda3/envs/week10/lib/python3.8/site-packages/certifi/cacert.pem'
2024-03-21 15:51:56,545 - DEBUG - load_ssl_context verify=True cert=None trust_env=True http2=False
2024-03-21 15:51:56,546 - DEBUG - load_verify_locations cafile='/home/babi/miniconda3/envs/week10/lib/python3.8/site-packages/certifi/cacert.pem'


In [3]:
rag_chain = Rag()

2024-03-21 15:51:57,428 - DEBUG - load_ssl_context verify=True cert=None trust_env=True http2=False
2024-03-21 15:51:57,429 - DEBUG - load_verify_locations cafile='/home/babi/miniconda3/envs/week10/lib/python3.8/site-packages/certifi/cacert.pem'
2024-03-21 15:51:57,439 - DEBUG - load_ssl_context verify=True cert=None trust_env=True http2=False
2024-03-21 15:51:57,440 - DEBUG - load_verify_locations cafile='/home/babi/miniconda3/envs/week10/lib/python3.8/site-packages/certifi/cacert.pem'
2024-03-21 15:51:57,642 - DEBUG - Registering provider: token_config
2024-03-21 15:51:57,644 - DEBUG - Registering provider: user_token_config
2024-03-21 15:51:57,646 - DEBUG - Registering provider: token
2024-03-21 15:51:57,647 - DEBUG - Registering provider: token
2024-03-21 15:51:57,657 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2024-03-21 15:51:57,766 - DEBUG - Starting component System
2024-03-21 15:51:57,766 - DEBUG - S

2024-03-21 15:51:58,278 - DEBUG - Starting new HTTPS connection (1): us-api.i.posthog.com:443
2024-03-21 15:51:59,320 - DEBUG - https://us-api.i.posthog.com:443 "POST /batch/ HTTP/1.1" 200 None


#### Populate Campaign Data into vector database

In [6]:
campaign_df = pd.read_csv("../data/campaign_data.csv")

In [7]:
campaign_df

,campaignid,budget_amount,budget_currencycode,KPI,pricing_model,geo_targeting,vertical,targeting
0,f4p5a01,277.12,USD,Engagement,CPE,NaN,Sports,Opinion Leaders| Investors| Policy Makers| C&I...
1,664kg61,160.76,USD,Engagement,CPE,NaN,Sports,NaN
2,cvpj79c,65.61,USD,Engagement,CPE,NaN,Sports,Opinion Leaders| Investors| Policy Makers| C&I...
3,kk6arx4,1300.00,USD,Engagement,CPE,NaN,Hobbies & Interests,NaN
4,fisff0r,1728.87,USD,Click,CPC,NaN,Automotive,NaN
...,...,...,...,...,...,...,...,...
81,5hvng9w,1826.47,USD,Reach,CPM,NaN,Arts & Entertainment,NaN
82,m1p7fvc,2331.00,USD,Engagement,CPE,NaN,Health & Fitness,25-35| Males (30%) & Females (70%)| LSM 6-16
83,woln7nu,8691.59,USD,Reach,CPM,NaN,Arts & Entertainment,NaN
84,9j2r3sb,1500.00,USD,Click,CPC,NaN,Hobbies & Interests,NaN


In [8]:
campaign_df.columns

Index(['campaignid', 'budget_amount', 'budget_currencycode', 'KPI',
       'pricing_model', 'geo_targeting', 'vertical', 'targeting'],
      dtype='object')